In [60]:
import os
import sys
from glob import glob
import yaml
import numpy as np
import torch
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import scienceplots
plt.style.use('science')
base_dir = os.path.join(os.getcwd(), '..')
sys.path.append(base_dir)

In [61]:
with open('../config/svgp.yaml', 'r') as f:
    cfg = yaml.safe_load(f)
    cfg['dataset']['path'] = os.path.join('../', cfg['dataset']['path'])
    cfg['model']['n_inducing_points'] = 60

## SVGP

In [82]:
import run_svgp_vertical_profile as main

model = main.make_model(cfg, data)

def extract_params_svgp(state_dict_path):
    state_dict = torch.load(state_dict_path, map_location='cpu')
    model.load_state_dict(state_dict)
    gamma = model.kernel.kernels[0].kernels[0].outputscale.detach().item()
    ltime = model.kernel.kernels[0].kernels[0].base_kernel.lengthscale[0][0].detach().item()
    llatlon = model.kernel.kernels[0].kernels[1].lengthscale[0][0].detach().item()
    lmeteo = model.kernel.kernels[1].lengthscale[0].detach()
    lT, lP, lRH, lomega = lmeteo
    lT, lP, lRH, lomega = lT.item(), lP.item(), lRH.item(), lomega.item()
    columns = [r'$\gamma_1$', r'$\ell_t$', r'$\ell_{latlon}$',
               r'$\ell_P$', r'$\ell_T$', r'$\ell_{RH}$', r'$\ell_\omega$']
    params_vals = [[gamma, ltime, llatlon, lP, lT, lRH, lomega]]
    params_df = pd.DataFrame(data=params_vals, columns=columns)
    return params_df

In [83]:
svgp_paths = glob('../experiments/data/outputs/svgp/seed_*')
dfs = []
for path in svgp_paths:
    params_df = extract_params_svgp(os.path.join(path, 'state_dict.pt'))
    dfs.append(params_df)
params_df = pd.concat(dfs, axis=0).reset_index(drop=True)
params_df.aggregate(['mean', 'std'])

,$\gamma_1$,$\ell_t$,$\ell_{latlon}$,$\ell_P$,$\ell_T$,$\ell_{RH}$,$\ell_\omega$
mean,0.383675,6.137288,1.440151,2.141153,3.189153,3.405175,7.860105
std,0.042412,0.285132,0.057706,0.095953,0.166251,0.213173,0.294995


## GP only

In [105]:
import ablations.run_svgp_gp_only as main

model = main.make_model(cfg, data)

def extract_params_svgp(state_dict_path):
    state_dict = torch.load(state_dict_path, map_location='cpu')
    model.load_state_dict(state_dict)
    gamma = model.kernel.kernels[0].kernels[0].outputscale.detach().item()
    ltime = model.kernel.kernels[0].kernels[0].base_kernel.lengthscale[0][0].detach().item()
    llatlon = model.kernel.kernels[0].kernels[1].lengthscale[0][0].detach().item()
    lmeteo = model.kernel.kernels[1].lengthscale[0].detach()
    lT, lP, lRH, lomega = lmeteo
    lT, lP, lRH, lomega = lT.item(), lP.item(), lRH.item(), lomega.item()
    columns = [r'$\gamma_1$', r'$\ell_t$', r'$\ell_{latlon}$',
               r'$\ell_P$', r'$\ell_T$', r'$\ell_{RH}$', r'$\ell_\omega$']
    params_vals = [[gamma, ltime, llatlon, lP, lT, lRH, lomega]]
    params_df = pd.DataFrame(data=params_vals, columns=columns)
    return params_df

In [106]:
svgp_paths = glob('../experiments/data/outputs/ablation/svgp-gp-only/seed_*')
dfs = []
for path in svgp_paths:
    params_df = extract_params_svgp(os.path.join(path, 'state_dict.pt'))
    dfs.append(params_df)
params_df = pd.concat(dfs, axis=0).reset_index(drop=True)
params_df.aggregate(['mean', 'std'])

,$\gamma_1$,$\ell_t$,$\ell_{latlon}$,$\ell_P$,$\ell_T$,$\ell_{RH}$,$\ell_\omega$
mean,0.427220,5.624494,1.439399,2.121922,3.939295,3.731438,9.057024
std,0.047031,0.215338,0.057644,0.114327,0.234382,0.307573,0.618278


## Spatiotemporal only

In [110]:
import ablations.run_svgp_spatiotemporal_only as main

model = main.make_model(cfg, data)

def extract_params_svgp(state_dict_path):
    state_dict = torch.load(state_dict_path, map_location='cpu')
    model.load_state_dict(state_dict)
    gamma = np.nan
    ltime = model.kernel.kernels[0].lengthscale[0][0].detach().item()
    llatlon = model.kernel.kernels[1].lengthscale[0][0].detach().item()
    lT, lP, lRH, lomega = np.nan, np.nan, np.nan, np.nan
    columns = [r'$\gamma_1$', r'$\ell_t$', r'$\ell_{latlon}$',
               r'$\ell_P$', r'$\ell_T$', r'$\ell_{RH}$', r'$\ell_\omega$']
    params_vals = [[gamma, ltime, llatlon, lP, lT, lRH, lomega]]
    params_df = pd.DataFrame(data=params_vals, columns=columns)
    return params_df

In [111]:
svgp_paths = glob('../experiments/data/outputs/ablation/svgp-spatiotemporal-only/seed_*')
dfs = []
for path in svgp_paths:
    params_df = extract_params_svgp(os.path.join(path, 'state_dict.pt'))
    dfs.append(params_df)
params_df = pd.concat(dfs, axis=0).reset_index(drop=True)
params_df.aggregate(['mean', 'std']).fillna('-')

,$\gamma_1$,$\ell_t$,$\ell_{latlon}$,$\ell_P$,$\ell_T$,$\ell_{RH}$,$\ell_\omega$
mean,-,5.839467,1.242507,-,-,-,-
std,-,0.876944,0.062819,-,-,-,-


## Meteorological only

In [117]:
import ablations.run_svgp_meteorological_only as main

model = main.make_model(cfg, data)

def extract_params_svgp(state_dict_path):
    state_dict = torch.load(state_dict_path, map_location='cpu')
    model.load_state_dict(state_dict)
    gamma = np.nan
    ltime = np.nan
    llatlon = np.nan
    lT, lP, lRH, lomega = model.kernel.lengthscale[0].detach()
    lT, lP, lRH, lomega = lT.item(), lP.item(), lRH.item(), lomega.item()
    columns = [r'$\gamma_1$', r'$\ell_t$', r'$\ell_{latlon}$',
               r'$\ell_P$', r'$\ell_T$', r'$\ell_{RH}$', r'$\ell_\omega$']
    params_vals = [[gamma, ltime, llatlon, lP, lT, lRH, lomega]]
    params_df = pd.DataFrame(data=params_vals, columns=columns)
    return params_df

In [118]:
svgp_paths = glob('../experiments/data/outputs/ablation/svgp-meteorological-only/seed_*')
dfs = []
for path in svgp_paths:
    params_df = extract_params_svgp(os.path.join(path, 'state_dict.pt'))
    dfs.append(params_df)
params_df = pd.concat(dfs, axis=0).reset_index(drop=True)
params_df.aggregate(['mean', 'std']).fillna('-')

,$\gamma_1$,$\ell_t$,$\ell_{latlon}$,$\ell_P$,$\ell_T$,$\ell_{RH}$,$\ell_\omega$
mean,-,-,-,1.696700,3.302550,2.929692,6.812116
std,-,-,-,0.048153,0.086488,0.119935,0.230458


## Product kernel

In [130]:
import ablations.run_svgp_product_kernel as main

model = main.make_model(cfg, data)

def extract_params_svgp(state_dict_path):
    state_dict = torch.load(state_dict_path, map_location='cpu')
    model.load_state_dict(state_dict)
    gamma = model.kernel.kernels[0].outputscale.detach().item()
    ltime = model.kernel.kernels[0].base_kernel.lengthscale[0][0].detach().item()
    llatlon = model.kernel.kernels[1].lengthscale[0][0].detach().item()
    lT = model.kernel.kernels[2].lengthscale[0][0].detach().item()
    lP = model.kernel.kernels[3].lengthscale[0][0].detach().item()
    lRH = model.kernel.kernels[4].lengthscale[0][0].detach().item()
    lomega = model.kernel.kernels[5].lengthscale[0][0].detach().item()
    columns = [r'$\gamma_1$', r'$\ell_t$', r'$\ell_{latlon}$',
               r'$\ell_P$', r'$\ell_T$', r'$\ell_{RH}$', r'$\ell_\omega$']
    params_vals = [[gamma, ltime, llatlon, lP, lT, lRH, lomega]]
    params_df = pd.DataFrame(data=params_vals, columns=columns)
    return params_df

In [131]:
svgp_paths = glob('../experiments/data/outputs/ablation/svgp-product-kernel/seed_*')
dfs = []
for path in svgp_paths:
    params_df = extract_params_svgp(os.path.join(path, 'state_dict.pt'))
    dfs.append(params_df)
params_df = pd.concat(dfs, axis=0).reset_index(drop=True)
params_df.aggregate(['mean', 'std']).fillna('-')

,$\gamma_1$,$\ell_t$,$\ell_{latlon}$,$\ell_P$,$\ell_T$,$\ell_{RH}$,$\ell_\omega$
mean,0.922213,8.369800,1.560460,3.906698,5.406649,5.252014,8.092106
std,0.025252,0.107463,0.001383,0.313308,0.211576,0.165437,0.124011


## Additive kernel

In [136]:
import ablations.run_svgp_additive_kernel as main

model = main.make_model(cfg, data)

def extract_params_svgp(state_dict_path):
    state_dict = torch.load(state_dict_path, map_location='cpu')
    model.load_state_dict(state_dict)
    gammatime = model.kernel.kernels[0].outputscale.detach().item()
    ltime = model.kernel.kernels[0].base_kernel.lengthscale[0][0].detach().item()
    
    gammalatlon = model.kernel.kernels[1].outputscale.detach().item()
    llatlon = model.kernel.kernels[1].base_kernel.lengthscale[0][0].detach().item()
    
    gammaT = model.kernel.kernels[2].outputscale.detach().item()
    lT = model.kernel.kernels[2].base_kernel.lengthscale[0][0].detach().item()
    
    gammaP = model.kernel.kernels[3].outputscale.detach().item()
    lP = model.kernel.kernels[3].base_kernel.lengthscale[0][0].detach().item()

    gammaRH = model.kernel.kernels[4].outputscale.detach().item()
    lRH = model.kernel.kernels[4].base_kernel.lengthscale[0][0].detach().item()
    
    gammaomega = model.kernel.kernels[5].outputscale.detach().item()
    lomega = model.kernel.kernels[5].base_kernel.lengthscale[0][0].detach().item()
    
    columns = [r'$\gamma_t$', r'$\gamma_{latlon}$', r'$\gamma_T$', r'$\gamma_P$', r'$\gamma_{RH}$', r'$\gamma_\omega$',
               r'$\ell_t$', r'$\ell_{latlon}$', r'$\ell_P$', r'$\ell_T$', r'$\ell_{RH}$', r'$\ell_\omega$']
    params_vals = [[gammatime, gammalatlon, gammaT, gammaP, gammaRH, gammaomega,
                    ltime, llatlon, lP, lT, lRH, lomega]]
    params_df = pd.DataFrame(data=params_vals, columns=columns)
    return params_df

In [137]:
svgp_paths = glob('../experiments/data/outputs/ablation/svgp-additive-kernel/seed_*')
dfs = []
for path in svgp_paths:
    params_df = extract_params_svgp(os.path.join(path, 'state_dict.pt'))
    dfs.append(params_df)
params_df = pd.concat(dfs, axis=0).reset_index(drop=True)
params_df.aggregate(['mean', 'std']).fillna('-')

,$\gamma_t$,$\gamma_{latlon}$,$\gamma_T$,$\gamma_P$,$\gamma_{RH}$,$\gamma_\omega$,$\ell_t$,$\ell_{latlon}$,$\ell_P$,$\ell_T$,$\ell_{RH}$,$\ell_\omega$
mean,1.363341,0.340111,0.312388,0.730937,0.229155,0.110334,5.630867,1.113565,3.312030,2.958025,2.963843,3.34290
std,0.143512,0.086149,0.054823,0.094799,0.029682,0.015209,0.661336,0.140317,0.202985,0.174606,0.183611,0.17392
